### Все выгрузки, описанные ниже необходимо делать за 2017 год. 
### Все данные связаны с продуктом "Туры"

Импортировать модули

In [1]:
from elasticsearch import Elasticsearch
import elasticsearch
from elasticsearch_dsl import Search, Q, A

import time
import datetime as dt
import pandas as pd
import numpy as np

client = Elasticsearch(hosts=["http://analytics-3-6.elc.tutu.ru:9200"], timeout=120)

Функция перевода времени из timestamp в человеческий вид и функция для обратной процедуры

In [2]:
def from_timestamp(t):
    t = int(t/1000)
    return dt.datetime.fromtimestamp(t).date()



def parseTime (strTime):
    timeFormat = '%d.%m.%Y %H:%M:%S'
    return int(time.mktime(time.strptime(strTime, timeFormat)) * 1000)

### Вывести кол-во людей, которые искали туры на выдаче с вылетами из городов

* Страница выдачи: tours_search_list
* Какое поле в tours_search_list отвечает за город вылета надо найти (Кибана и спейс Userway вам в помощь)
* Города вылета: Москва, Санкт-Петербург, Новосибирск, Екатеринбург. Какие id соответсвуют городам можете найти в blackbox, раздел avia, таблица rm_city
* Выбираем только реальных людей, поле is_real: true (а где это лежит?)
* Написать запрос двумя способами: с query_string и без него
* Данные за период: июнь 2017

In [ ]:
Ответ: 218801

Решение без query_string

In [3]:
dt_str = ['01.06.2017 00:00:00', '30.06.2017 23:59:59']
date = [parseTime(i) for i in dt_str]
s_e = Search(using=client).index('userway_alias').doc_type('event')
s_c = Search(using=client).index('userway_alias').doc_type('client')

In [189]:
dt_str = ['2017-06-01 00:00:00', '2017-06-30 23:59:59']
cities_query_1 = s_e.query(Q('prefix', title='tours_search_list')&
                                    Q('range', time={'gte':dt_str[0], 'lte':dt_str[1], 
                                                    'format':'yyyy-MM-dd HH:mm:ss', 'time_zone':'+03:00'})&
                                    Q('terms', departure_city_id=[491,75,58,29])&
                                    Q('has_parent', query=
                                      Q('has_child', query=
                                        Q('prefix', is_real='True'), 
                                                        type='client'), 
                                                            type='session'))

In [190]:
cities_query_1.aggs.bucket('id', 'terms', field ='_parent', size=0)

Terms(field='_parent', size=0)

In [191]:
cities_1 = cities_query_1.execute()

In [192]:
#ответ 218801 получится,если в запросе убрать условие на is_real 
len(cities_1.aggregations.id.buckets) 

218798

Решение с query_string

In [ ]:
cities_query_2 = s_e.query(Q('query_string', query=
                                    'title:%s AND time:[%s TO %s] AND departure_city_id: (%s OR %s OR %s OR %s)' % (
                                        'tours_search_list', date[0], date[1],'491', '75','58','29'))&
                         Q('has_parent', query=
                              Q('has_child', query=
                                       Q('query_string', query= 'is_real:%s' %('True')) ,type='client'),type='session'))

cities_2 = cities_query_2.execute()
cities_2.hits.total

### Выгрузить данные из Userway c помощью scan

* Выгрузить все страницы раздела туров за июнь, которые посещали покупатели туров за июнь. 
  Перечислять страницы вручную не надо, необходимо использовать регулярное выражение
* Поля в выгрузке: сессия, название страницы, версия сайта, время посещения страницы
* Не использовать query_string

Кол-во строк в выгрузке: 79830

Решение

In [4]:
dt_str = ['2017-06-01 00:00:00', '2017-06-30 23:59:59']
s_e = Search(using=client).index('userway_alias').doc_type('event')
tours_query = s_e.query(Q('range', time={'gte':dt_str[0], 'lte':dt_str[1], 
                                                    'format':'yyyy-MM-dd HH:mm:ss', 'time_zone':'+03:00'})&
                        Q('regexp', title='tours_.*')
                        )

In [5]:
with open('tours.csv','w') as file:
    for res in tours_query.scan():
        res_d = res.to_dict()
        print('\t'.join(map(str,[res.meta.parent, res.title, 
                                 res.site_version, res.time,
                                 res_d.get('profit', 'None')])),file=file)

In [24]:
data_tours=pd.read_csv('tours.csv', sep='\t', names=[ 'session_id','title','site_version','time','non'])

In [25]:
data_tours=data_tours[['session_id','title','site_version','time']]

In [29]:
data_tours['time'].map(from_timestamp)

0          2017-06-01
1          2017-06-01
2          2017-06-01
3          2017-06-01
4          2017-06-13
5          2017-06-10
6          2017-06-13
7          2017-06-10
8          2017-06-01
9          2017-06-13
10         2017-06-01
11         2017-06-10
12         2017-06-13
13         2017-06-01
14         2017-06-01
15         2017-06-01
16         2017-06-10
17         2017-06-01
18         2017-06-01
19         2017-06-13
20         2017-06-13
21         2017-06-01
22         2017-06-13
23         2017-06-01
24         2017-06-13
25         2017-06-10
26         2017-06-01
27         2017-06-13
28         2017-06-13
29         2017-06-10
              ...    
8746829    2017-06-09
8746830    2017-06-09
8746831    2017-06-09
8746832    2017-06-09
8746833    2017-06-09
8746834    2017-06-09
8746835    2017-06-09
8746836    2017-06-09
8746837    2017-06-09
8746838    2017-06-09
8746839    2017-06-09
8746840    2017-06-09
8746841    2017-06-09
8746842    2017-06-09
8746843   

In [30]:
data_tours.head()

,session_id,title,site_version,time
0,06010617-d026-4751-9150-3475f078b27b,tours_hotel_list,full,1496325907000
1,06010617-d026-4751-9150-3475f078b27b,tours_filter_changed,full,1496325904000
2,3c060217-f338-4695-b5fd-bd93ccfe5401,tours_feedback_read_more_click,full,1496325902000
3,3c060217-f338-4695-b5fd-bd93ccfe5401,tours_feedback_read_more_click,full,1496325907000
4,d8100116-d0bd-48d4-beb7-d8f5ccb25ab2,tours_search_list,full,1497354597000


### Сделать агрегацию по дням для посетителей страницы отеля для десктопной версии

* Страница отеля title:tours_hotel
* Версия сайта: site_version, а что именно надо написать необходимо найти (Кибана и спейс Userway вам в помощь)
* Выбираем только реальных людей, поле is_real: true
* Обратите внимание, что нас интересуют пользователи страницы, а не посещения страницы
* Данные за период: 01.06.2017 - 14.06.2017
* Использовать query_string

Решение

In [43]:
dt_str_1 = ['01.06.2017 00:00:00', '14.06.2017 23:59:59']
date_1 = [parseTime(i) for i in dt_str_1]

In [48]:
schedule_desktop_query = s_e.query(Q('query_string', query=
                                    'title:%s AND time:[%s TO %s] AND site_version:%s' % (
                                        'tours_hotel', date_1[0], date_1[1],'full'))&
                                    Q('has_parent', query=
                                       Q('has_child', query=
                                           Q('query_string', query= 'is_real:%s' %('True')) ,
                                                  type='client'),
                                                      type='session'))

In [49]:
schedule_desktop_query.aggs.bucket('per_day', 'date_histogram', field='time', interval='day', time_zone = 'Europe/Moscow')\
               .metric('visitors','cardinality', field = '_parent')

DateHistogram(aggs={'visitors': Cardinality(field='_parent')}, field='time', interval='day', time_zone='Europe/Moscow')

In [50]:
schedule_desktop = schedule_desktop_query.execute()

In [51]:
schedule_desktop.to_dict()

{'took': 43182,
 'timed_out': False,
 '_shards': {'total': 144, 'successful': 144, 'failed': 0},
 'hits': {'total': 345891,
  'max_score': 7.748,
  'hits': [{'_index': 'userway_alpha',
    '_type': 'event',
    '_id': 'AVyQwnqK05tmPuuT0ZfE',
    '_score': 7.748,
    '_routing': 'c5100617-91b9-4989-b48e-8ba9bf9a5bc5',
    '_parent': 'c5100617-91b9-4989-b48e-8ba9bf9a5bc5',
    '_source': {'arrival_country_id': 197,
     'page_id': '5kplP7OPtpm',
     'zone': 'main',
     'site_version': 'full',
     'hotel_id': 64130,
     'ip': '93.81.43.97',
     'time': 1497077207000,
     'type': 'page',
     'title': 'tours_hotel'}},
   {'_index': 'userway_alpha',
    '_type': 'event',
    '_id': 'AVyQoKZ-05tmPuuTyASa',
    '_score': 7.748,
    '_routing': 'af100517-daa7-4237-8fec-f229e1c3b6db',
    '_parent': 'af100517-daa7-4237-8fec-f229e1c3b6db',
    '_source': {'arrival_country_id': 97,
     'departure_city_id': 491,
     'child_count': 0,
     'site_version': 'full',
     'hotel_id': 5023,
    

In [52]:
print('date', 'visitors')
for i in schedule_desktop.aggregations.per_day.buckets:
    print(from_timestamp(i.key),i.visitors.value)  

date visitors
2017-06-01 9243
2017-06-02 8167
2017-06-03 6598
2017-06-04 8161
2017-06-05 10484
2017-06-06 9652
2017-06-07 9533
2017-06-08 9678
2017-06-09 6776
2017-06-10 4727
2017-06-11 5038
2017-06-12 4539
2017-06-13 8002
2017-06-14 9743


### Сделать агрегацию по странам, которые искали на выдаче с вылетом из Москвы

* Страница title: tours_search_list
* Выбираем только реальных людей, поле is_real: true
* Еще интересуют поиски, в которых не применялся фильтр "Питание". За это отвечает параметр tours_filter_meal в tours_search_list. Если фильтр применяли, параметр пишется, если нет, то параметр не пишется (там не проставляется значение NULL)
* Какие поля в tours_search_list отвечают за город вылета и страну прилета надо найти (Кибана и спейс Userway вам в помощь)
* Данные за период: 01.06.2017 - 14.06.2017
* При написании запросов не использовать query_string
* Время не переводим в timestamp для запросов, в человеческом виде передаем в запросах, не забываем про часовой пояс

Решение

In [53]:
dt_str_1 = ['2017-06-01 00:00:00', '2017-06-14 23:59:59']

In [60]:
schedule_msk_query = s_e.query(Q('term', title='tours_search_list')&
                                     ~Q('exists',field='tours_filter_meal')&
                                     Q('range', time={'gte':dt_str_1[0], 'lte':dt_str_1[1], 
                                                    'format':'yyyy-MM-dd HH:mm:ss', 'time_zone':'+03:00'})&
                                     Q('term', departure_city_id=491)&
                                     Q('has_parent', query=
                                       Q('has_child', query=
                                         Q('term', is_real='True'), 
                                        type='client'), 
                                     type='session'))

In [61]:
schedule_msk_query.aggs.bucket('arr_stations', 'terms', field ='arrival_country_id', size=0)

Terms(field='arrival_country_id', size=0)

In [62]:
schedule_msk = schedule_msk_query.execute()

In [63]:
schedule_msk.to_dict()

{'took': 17792,
 'timed_out': False,
 '_shards': {'total': 144, 'successful': 144, 'failed': 0},
 'hits': {'total': 250209,
  'max_score': 8.802486,
  'hits': [{'_index': 'userway_alpha',
    '_type': 'event',
    '_id': 'AVyHPHB_05tmPuuTLmsq',
    '_score': 8.802486,
    '_routing': '4e301215-010a-41ff-9a67-df4e53072aeb',
    '_parent': '4e301215-010a-41ff-9a67-df4e53072aeb',
    '_source': {'arrival_country_id': 197,
     'departure_city_id': 491,
     'child_count': 0,
     'site_version': 'mobile',
     'ip': '83.220.236.21',
     'resort_id': 267,
     'tours_offers_count': 11,
     'max_nights': 11,
     'search_date_begin': 1505250000000,
     'type': 'page',
     'title': 'tours_search_list',
     'page_id': '5kn118exYwh',
     'zone': 'main',
     'time': 1496917462000,
     'page': 'tours_search',
     'search_date_timestamp': 1505509200000,
     'min_nights': 7,
     'adult_count': 2,
     'input_nights': 9,
     'search_date_end': 1505768400000}},
   {'_index': 'userway_alp

In [79]:
print('country_id', 'search')
j=0
for i in schedule_msk.aggregations.arr_stations.buckets:
    print("%1d%7d%7d"%(j, i.key, i.doc_count))
    j=j+1

country_id search
0      7  67382
1    197  46761
2     64  20846
3    188  13812
4     95  12073
5    195  11914
6     84   9948
7    263   7609
8     85   7355
9    268   6914
10     32   6295
11     48   5904
12     71   5308
13    145   3010
14    213   2866
15     76   2449
16    106   1991
17    127   1911
18     65   1725
19     97   1586
20     72   1561
21     77   1517
22    125   1396
23     78   1230
24     79    941
25    217    846
26    207    845
27    210    843
28    161    757
29     58    512
30    129    511
31    126    450
32     20    328
33    264    241
34     27    202
35    215    180
36     11    161
37     16     29


### Сделать агрегацию поисков на странице отеля по дате и по диапазону дат вылета за июнь

* Левая и правая границы начала тура соотвественно: search_date_begin и search_date_end, 
    интересует агрегация по всевозможным разницам между датами
* Учитываем реальных людей
* Можно использовать query_string, ограничений нет

** Внимание! Данные не выгружаем, разницу дат получаем уже в запросе к Эластику**

Решение

In [27]:
dt_str = ['01.06.2017 00:00:00', '30.06.2017 23:59:59']
date = [parseTime(i) for i in dt_str]

In [32]:
hotels_query = s_e.query(Q('query_string', query=
                                    'title:%s AND time:[%s TO %s]' % (
                                        'tours_hotel', date[0], date[1])))

In [33]:
hotels_query.aggs.bucket('per_day', 'date_histogram', field='time', interval='day', time_zone = 'Europe/Moscow').\
               bucket('diff', 'terms', 
                script = "(doc['search_date_end'].value - doc['search_date_begin'].value).intdiv(86400000)", size = 100000)


Terms(script="(doc['search_date_end'].value - doc['search_date_begin'].value).intdiv(86400000)", size=100000)

In [34]:
hotels = hotels_query.execute()

In [52]:
print('cdate','delta','search')
i=0
k=0
for i in range (30):
    slov=hotels.aggregations.per_day.buckets[i].to_dict()
    j=0
    for j in range(len(slov['diff']['buckets'])) :
        print(k, str(from_timestamp(slov['key'])),
               slov['diff']['buckets'][j]['key'],
                slov['diff']['buckets'][j]['doc_count'])
        k=k+1

cdate delta search
0 2017-06-01 6 18775
1 2017-06-01 0 5900
2 2017-06-01 3 1806
3 2017-06-01 30 1597
4 2017-06-01 27 1033
5 2017-06-01 5 699
6 2017-06-01 29 506
7 2017-06-01 12 252
8 2017-06-01 4 164
9 2017-06-01 7 147
10 2017-06-01 2 90
11 2017-06-01 1 62
12 2017-06-01 11 42
13 2017-06-01 10 34
14 2017-06-01 9 34
15 2017-06-01 8 30
16 2017-06-01 89 18
17 2017-06-01 13 15
18 2017-06-01 14 10
19 2017-06-01 26 9
20 2017-06-01 -1 8
21 2017-06-01 -3 8
22 2017-06-01 18 8
23 2017-06-01 15 7
24 2017-06-01 16 7
25 2017-06-01 -2 6
26 2017-06-01 -6 6
27 2017-06-01 21 6
28 2017-06-01 -9 5
29 2017-06-01 47 5
30 2017-06-01 -11 4
31 2017-06-01 -20 4
32 2017-06-01 17 4
33 2017-06-01 22 4
34 2017-06-01 23 4
35 2017-06-01 25 4
36 2017-06-01 28 4
37 2017-06-01 156 2
38 2017-06-01 19 2
39 2017-06-01 31 2
40 2017-06-01 34 2
41 2017-06-01 35 2
42 2017-06-01 39 2
43 2017-06-01 40 2
44 2017-06-01 49 2
45 2017-06-01 87 2
46 2017-06-01 -10 1
47 2017-06-01 -103 1
48 2017-06-01 -12 1
49 2017-06-01 -13 1
50 2017-

667 2017-06-10 -26 2
668 2017-06-10 -28 2
669 2017-06-10 24 2
670 2017-06-10 27 2
671 2017-06-10 36 2
672 2017-06-10 49 2
673 2017-06-10 -104 1
674 2017-06-10 -13 1
675 2017-06-10 -154 1
676 2017-06-10 -167 1
677 2017-06-10 -17 1
678 2017-06-10 -170 1
679 2017-06-10 -21 1
680 2017-06-10 -237 1
681 2017-06-10 -25 1
682 2017-06-10 -29 1
683 2017-06-10 -319 1
684 2017-06-10 -356 1
685 2017-06-10 -39 1
686 2017-06-10 -42 1
687 2017-06-10 -96 1
688 2017-06-10 16 1
689 2017-06-10 19 1
690 2017-06-10 21 1
691 2017-06-10 31 1
692 2017-06-10 51 1
693 2017-06-11 6 14648
694 2017-06-11 0 3706
695 2017-06-11 30 1345
696 2017-06-11 3 957
697 2017-06-11 5 570
698 2017-06-11 29 440
699 2017-06-11 17 427
700 2017-06-11 12 122
701 2017-06-11 4 102
702 2017-06-11 7 63
703 2017-06-11 2 33
704 2017-06-11 10 27
705 2017-06-11 -3 16
706 2017-06-11 8 16
707 2017-06-11 -2 15
708 2017-06-11 9 15
709 2017-06-11 -1 13
710 2017-06-11 1 12
711 2017-06-11 11 12
712 2017-06-11 14 9
713 2017-06-11 28 6
714 2017-06-11

1256 2017-06-19 -334 1
1257 2017-06-19 -34 1
1258 2017-06-19 -383 1
1259 2017-06-19 -39 1
1260 2017-06-19 -394 1
1261 2017-06-19 -4 1
1262 2017-06-19 -81 1
1263 2017-06-19 -99 1
1264 2017-06-19 20 1
1265 2017-06-19 21 1
1266 2017-06-19 24 1
1267 2017-06-19 31 1
1268 2017-06-19 33 1
1269 2017-06-19 35 1
1270 2017-06-19 50 1
1271 2017-06-20 6 22866
1272 2017-06-20 0 6779
1273 2017-06-20 30 2432
1274 2017-06-20 3 2130
1275 2017-06-20 5 920
1276 2017-06-20 29 864
1277 2017-06-20 8 477
1278 2017-06-20 12 222
1279 2017-06-20 4 186
1280 2017-06-20 70 121
1281 2017-06-20 2 94
1282 2017-06-20 7 82
1283 2017-06-20 1 41
1284 2017-06-20 10 33
1285 2017-06-20 11 25
1286 2017-06-20 9 22
1287 2017-06-20 -2 19
1288 2017-06-20 14 19
1289 2017-06-20 13 13
1290 2017-06-20 45 10
1291 2017-06-20 65 9
1292 2017-06-20 -1 8
1293 2017-06-20 -21 6
1294 2017-06-20 41 6
1295 2017-06-20 17 5
1296 2017-06-20 21 5
1297 2017-06-20 27 5
1298 2017-06-20 -20 4
1299 2017-06-20 -3 4
1300 2017-06-20 18 4
1301 2017-06-20 -1

1905 2017-06-29 -383 1
1906 2017-06-29 -4 1
1907 2017-06-29 -45 1
1908 2017-06-29 -578 1
1909 2017-06-29 -617 1
1910 2017-06-29 -7 1
1911 2017-06-29 -9 1
1912 2017-06-29 18 1
1913 2017-06-29 19 1
1914 2017-06-29 20 1
1915 2017-06-29 22 1
1916 2017-06-29 43 1
1917 2017-06-29 52 1
1918 2017-06-29 55 1
1919 2017-06-29 60 1
1920 2017-06-29 65 1
1921 2017-06-30 6 19954
1922 2017-06-30 0 5246
1923 2017-06-30 3 1924
1924 2017-06-30 5 1872
1925 2017-06-30 29 1605
1926 2017-06-30 30 1166
1927 2017-06-30 4 347
1928 2017-06-30 2 125
1929 2017-06-30 7 91
1930 2017-06-30 12 53
1931 2017-06-30 1 38
1932 2017-06-30 10 34
1933 2017-06-30 8 24
1934 2017-06-30 9 18
1935 2017-06-30 14 17
1936 2017-06-30 -1 16
1937 2017-06-30 -2 14
1938 2017-06-30 11 11
1939 2017-06-30 28 11
1940 2017-06-30 60 9
1941 2017-06-30 13 8
1942 2017-06-30 15 8
1943 2017-06-30 27 7
1944 2017-06-30 -4 5
1945 2017-06-30 -5 5
1946 2017-06-30 -21 4
1947 2017-06-30 -3 4
1948 2017-06-30 -6 3
1949 2017-06-30 21 3
1950 2017-06-30 -12 2
1